In [18]:
# from __future__ import print_function
# import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import log_loss
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop, Adam
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
import tensorflow as tf
from keras import backend as K
import keras
from keras import regularizers
from collections import Counter
# %matplotlib inline
np.random.seed(1671) # for reproducibility
# from help_function import LoadData

In [28]:
data = pd.read_csv("nn_all_data.csv")
train_data = data[data['mt']<25]
test_data = data[data['mt']>=25]

In [21]:
columns = list(train.columns)


In [23]:
print(train_x.shape)
print(test.shape)

(31680, 38)
(5280, 38)


In [26]:
train_x.head()

,regYear,regMonth,shift_model_adcode_mt_label_1,shift_model_adcode_mt_label_2,shift_model_adcode_mt_label_3,shift_model_adcode_mt_label_4,shift_model_adcode_mt_label_5,shift_model_adcode_mt_label_6,shift_model_adcode_mt_label_7,shift_model_adcode_mt_label_8,...,shift_model_adcode_mt_popularity_12,differ2_popularity_1,differ2_popularity_2,differ2_popularity_3,windows4_var,windows5_var,windows12_var,adcode,bodyType,model
0,2016,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,0,0
1,2016,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,20,0,0
2,2016,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,3,0,0
3,2016,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0
4,2016,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,19,0,0


In [31]:



## load data
# train_data = pd.read_csv('data/train.csv')
# test_data = pd.read_csv('data/test.csv')
epochs = 10
batch_size = 1024
classes = 1

## category feature one_hot
test_data['label'] = -1
data = pd.concat([train_data, test_data])
cate_feature =['adcode','bodyType','model','regYear',
 'regMonth']
for item in cate_feature:
    data[item] = LabelEncoder().fit_transform(data[item])
#     item_dummies = pd.get_dummies(data[item])
#     item_dummies.columns = [item + str(i + 1) for i in range(item_dummies.shape[1])]
#     data = pd.concat([data, item_dummies], axis=1)
# data.drop(cate_feature,axis=1,inplace=True)

train = data[data['label'] != -1]
test = data[data['label'] == -1]

##Clean up the memory
# del data, train_data, test_data
# gc.collect()

## get train feature
# del_feature = [ 
#  'id',
#  'model_adcode',]
# features = [i for i in train.columns if i not in del_feature]
features = ['regYear',
 'regMonth',
 'shift_model_adcode_mt_label_1',
 'shift_model_adcode_mt_label_2',
 'shift_model_adcode_mt_label_3',
 'shift_model_adcode_mt_label_4',
 'shift_model_adcode_mt_label_5',
 'shift_model_adcode_mt_label_6',
 'shift_model_adcode_mt_label_7',
 'shift_model_adcode_mt_label_8',
 'shift_model_adcode_mt_label_9',
 'shift_model_adcode_mt_label_10',
 'shift_model_adcode_mt_label_11',
 'shift_model_adcode_mt_label_12',
 'differ2_label_1',
 'differ2_label_2',
 'differ2_label_3',
 'shift_model_adcode_mt_popularity_1',
 'shift_model_adcode_mt_popularity_2',
 'shift_model_adcode_mt_popularity_3',
 'shift_model_adcode_mt_popularity_4',
 'shift_model_adcode_mt_popularity_5',
 'shift_model_adcode_mt_popularity_6',
 'shift_model_adcode_mt_popularity_7',
 'shift_model_adcode_mt_popularity_8',
 'shift_model_adcode_mt_popularity_9',
 'shift_model_adcode_mt_popularity_10',
 'shift_model_adcode_mt_popularity_11',
 'shift_model_adcode_mt_popularity_12',
 'differ2_popularity_1',
 'differ2_popularity_2',
 'differ2_popularity_3',
 'windows4_var',
 'windows5_var',
 'windows12_var',       
 'adcode',
 'bodyType',
 'model']

train_x = train[features]
train_y = train['label'].values
test = test[features]

## Fill missing value
for i in train_x.columns:
    # print(i, train_x[i].isnull().sum(), test[i].isnull().sum())
    if train_x[i].isnull().sum() != 0:
        train_x[i].fillna(0, inplace=True)
        test[i].fillna(0, inplace=True)

## normalized
scaler = StandardScaler()
train_X = scaler.fit_transform(train_x)
test_X = scaler.transform(test)



K.clear_session()
def MLP(dropout_rate=0.25, activation='relu'):
    start_neurons = 50
    model = Sequential()
    model.add(Dense(start_neurons, input_dim=train_X.shape[1], activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 2, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 4, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    model.add(Dense(start_neurons // 8, activation=activation))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate / 2))

    model.add(Dense(classes, activation='linear'))
    return model


def plot_loss_acc(history, fold):
    plt.plot(history.history['loss'][1:])
    plt.plot(history.history['val_loss'][1:])
    plt.title('model loss')
    plt.ylabel('val_loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'Validation'], loc='upper left')
#     plt.savefig('../../result/model_loss' + str(fold) + '.png')
    plt.show()

    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['train', 'Validation'], loc='upper left')
#     plt.savefig('../../result/model_accuracy' + str(fold) + '.png')
    plt.show()


folds = KFold(n_splits=5, shuffle=True, random_state=2019)
NN_predictions = np.zeros((test_X.shape[0], 1))
oof_preds = np.zeros((train_X.shape[0], 1))

patience = 50   ## How many steps to stop
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1,
                                        mode='auto', baseline=None)

for fold_, (trn_, val_) in enumerate(folds.split(train_x)):
    print("fold {}".format(fold_ + 1))
    x_train, y_train = train_X[trn_], train_y[trn_]
    x_valid, y_valid = train_X[val_], train_y[val_]


    model = MLP(dropout_rate=0.5, activation='relu')
    model.compile(optimizer='adam', loss='mse', metrics=['mae', 'acc'])
    history = model.fit(x_train, y_train,
                        validation_data=[x_valid, y_valid],
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[call_ES, ],
                        shuffle=True,
                        verbose=1)

    # plot_loss_acc(history, fold_ + 1)

    print('Loading Best Model')
    # # Get predicted probabilities for each class
    oof_preds[val_] = model.predict(x_valid, batch_size=batch_size)
    NN_predictions += model.predict(test_X, batch_size=batch_size) / folds.n_splits

print(NN_predictions)

D:\Users\X1Carbon\Anaconda3\envs\TensorFlow\lib\site-packages\pandas\core\generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


fold 1
Train on 25344 samples, validate on 6336 samples
Epoch 1/10
25344/25344 [==============================] - 3s 114us/step - loss: 997608.8277 - mean_absolute_error: 612.4324 - acc: 0.0000e+00 - val_loss: 893590.0909 - val_mean_absolute_error: 593.8396 - val_acc: 0.0000e+00
Epoch 2/10
25344/25344 [==============================] - 0s 18us/step - loss: 997063.8611 - mean_absolute_error: 612.3509 - acc: 0.0000e+00 - val_loss: 892979.6578 - val_mean_absolute_error: 593.7726 - val_acc: 0.0000e+00
Epoch 3/10
25344/25344 [==============================] - 0s 19us/step - loss: 996605.0114 - mean_absolute_error: 612.2654 - acc: 0.0000e+00 - val_loss: 892440.8535 - val_mean_absolute_error: 593.7504 - val_acc: 0.0000e+00
Epoch 4/10
25344/25344 [==============================] - 0s 17us/step - loss: 996124.7538 - mean_absolute_error: 612.1587 - acc: 0.0000e+00 - val_loss: 891849.2891 - val_mean_absolute_error: 593.7074 - val_acc: 0.0000e+00
Epoch 5/10
25344/25344 [===========================

KeyboardInterrupt: 